In [1]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import editdistance
import matplotlib.pyplot as plt
import copy

In [2]:
def get_transliteration(s_w):
    '''
    Transliterates using indic_transliteration, does a little postediting
    '''
    trans = transliterate(s_w, sanscript.DEVANAGARI, sanscript.OPTITRANS)
    trans = trans.rstrip('a')
    trans = trans.replace('M','n')
    trans = trans.replace('A', 'aa')
    trans = trans.lower()
    return trans
    
    

In [3]:
def is_trans(e_w, h_w):
    h_latin = get_transliteration(h_w)
    score = float(editdistance.eval(h_latin, e_w)/max(len(h_latin), len(e_w)))
    return score<=0.5

In [7]:
data = open("../data/post_processed_nums.txt", "r").read().split("\n\n")
new_data = open("../data/REW_corpus_no_tags.txt", "w")
trans_pairs = []

for lines in data:
#     print("lines: ", lines)
    analysis = lines.split('\n') 
#     print("analysis: ", analysis)
    if(len(analysis)<2):
        continue
    en_sent = analysis[0].split(' ')
    hin_sent = analysis[1].split(' ')
#     print("en sent: ", en_sent)
#     print("hin sent: ", hin_sent)
    new_en_sent = copy.copy(en_sent)
    new_hin_sent = copy.copy(hin_sent)
    for en_idx, line in enumerate(analysis[3:]):
#         print('en idx ', en_idx, " ", line)
        aligns = line.split("\t")
        if len(aligns)<=1:
            continue
        e_w = aligns[0]
        h_idx_list = [int(idx) for idx in aligns[1:]]
        for h_idx in h_idx_list:
            if(len(e_w)>3 and e_w!='this' and is_trans(e_w, hin_sent[h_idx])):
                trans_pairs.append((e_w, hin_sent[h_idx]))
#                 new_en_sent[en_idx] = "<REW>"+e_w+"</REW>"
#                 new_hin_sent[h_idx] = "<REW>"+e_w+"</REW>"
                new_en_sent[en_idx] = e_w
                new_hin_sent[h_idx] = e_w
    print(' '.join(new_en_sent)+"\t"+' '.join(new_hin_sent), file = new_data)
            

In [34]:
!wc -l data/REW_corpus.txt

  273885 data/REW_corpus.txt


In [9]:
!head -n 5 ../data/REW_corpus_tagged.txt

<REW>sharaabi</REW>	<REW>sharaabi</REW>
politicians do not have permission to do what needs to be done .	राजनीतिज्ञों के पास जो कार्य करना चाहिए , वह करने कि अनुमति नहीं है .
i 'd like to tell you about one such child ,	मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी ,
this percentage is even greater than the percentage in india .	यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
- <REW>john</REW> <REW>collins</REW>	- <REW>john</REW> <REW>collins</REW>
